In [105]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import pandas as pd
from tqdm import tqdm
import gensim
import pymorphy2
import functools as ft
from tqdm import tqdm
import string
import re

In [106]:
projects_new = np.load('vectors.npy')

In [107]:
xy = pd.read_csv('X.csv')

In [108]:
Y = list(xy['Y'])

In [109]:
X = xy.drop('Y', axis=1)

In [111]:
X

,names,age,projects,compet,lang
0,Смирнов Станислав Михайлович,17,Мне это интересно,Олимпиады по программированиюНе участвовал,"[2, 1]"
1,Коротков Даниил Александрович,16,В настоящее время занимаюсь разработкой игры н...,"Множество олимпиад по информатике, хакатон от ...",[1]
2,Кривошеев Марк Дмитриевич,14,Хочу поучаствовать в хакатоне так как давно хо...,Нет(Школа,"[10, 2, 11, 1]"
3,Волоси Виталий Михайлович,16,Участвовал в школьных олимпиадах по информатик...,Нет.Не участвовал,[]
4,Кузнецов Иван Вадимович,14,"До этого занимался активным изучением языка, п...",Постоянно принимаю участие в раундах Codeforce...,"[8, 1, 5]"
5,Гришина Полина Сергеевна,16,-,-Не участвовал,[7]
6,Анненков Владислав Алексеевич,15,"Реализовал несколько мелких игр вроде ""Тетрис""...",Участвовал только во всероссийских и московски...,"[8, 1]"
7,воскобойников илья сергеевич,16,создовал не сложные проекты на delphi,нетНе участвовал,[7]
8,Мусенко Анастасия Денисовна,15,хочу попробовать свои силы в данной области,нетНе участвовал,[7]
9,Никольский Даниил Романович,16,Проект виртуального ассистента управляемого го...,олимпиада ВМК МГУ\nШкола,"[4, 8, 1]"


In [47]:
def one_hot(arr):
    for i in range(len(arr)):
        langs = arr[i]
        oh_langs = [0 for i in range(13)]
        for lang in langs:
            oh_langs[lang] = 1
        arr[i] = oh_langs
    return arr

In [48]:
X['lang'] = list(map(lambda x: eval(x), list(X['lang'])))

In [49]:
X['lang'] = one_hot(list(X['lang']))

In [50]:
morph = pymorphy2.MorphAnalyzer()

In [51]:
def normalize_form(txt, morph):
    p = [morph.parse(x)[0].normal_form for x in txt]
    return p

In [52]:
def make_dict(x):
    dict_rus = ' '.join(x)
    translator = str.maketrans({key: ' ' for key in string.punctuation.replace('/', '').replace('-', '') + string.digits + '№'})
    dict_rus = dict_rus.translate(translator).lower()
    dict_rus_split = dict_rus.split()
    return dict_rus_split

In [53]:
def avg_feature_vector(words, model, num_features):
    featureVec = np.zeros((num_features,), dtype="float32")
    nwords = 0

    for word in words:
        nwords = nwords + 1
        try:
            featureVec = np.add(featureVec, model[word])
        except KeyError:
            featureVec = np.add(featureVec, np.zeros(num_features))

    if (nwords > 0):
        featureVec = np.divide(featureVec, nwords)
    return featureVec

In [54]:
X.fillna('', inplace=True)

In [55]:
w2v = gensim.models.KeyedVectors.load_word2vec_format(r'/home/anton/F2P/ruscorpora.model.bin', binary = True)

In [56]:
def words2vecs(arr, w2v, morth):
    arr = list(map(lambda x: x.split(), arr))
    arr = [normalize_form(make_dict(new), morth) for new in arr]
    arr = [avg_feature_vector(sent, w2v, 300) for sent in list(arr)]
    return arr

In [57]:
X['projects'] = words2vecs(list(X['projects']), w2v, morph)

In [58]:
X['compet'] = words2vecs(list(X['compet']), w2v, morph)

In [59]:
projects = pd.DataFrame(list(X['projects']))

In [18]:
projects_new = pd.DataFrame(projects_new)

NameError: name 'projects_new' is not defined

In [ ]:
projects_new.head()

In [60]:
for i in range(len(list(X['projects'])[0])):
    X['p' + str(i)] = projects[i]

In [61]:
X = X.drop('projects', axis=1)

In [62]:
compets  = pd.DataFrame(list(X['compet']))

In [63]:
for i in range(len(list(X['compet'])[0])):
    X['c' + str(i)] = compets[i]

In [64]:
X = X.drop('compet', axis=1)

In [65]:
langs = pd.DataFrame(list(X['lang']))

In [66]:
for i in range(len(list(X['lang'])[0])):
    X['l' + str(i)] = langs[i]

In [67]:
X = X.drop('lang', axis=1)

In [68]:
X = X.drop('names', axis=1)

In [69]:
X['age'] = list(map(lambda x: 12 if x == '12.5' or x == 'Егор' else int(re.findall('\d+', x)[0]), list(X['age'])))

In [70]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [71]:
X = X.drop('age', axis=1)

In [72]:
from sklearn.preprocessing import scale

In [97]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [98]:
grc = GridSearchCV(LogisticRegression(), {'C':list(range(1, 10))})
grc.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [1, 2, 3, 4, 5, 6, 7, 8, 9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [99]:
grc.best_params_

{'C': 6}

In [100]:
cv_res = grc.cv_results_

In [101]:
lg = LogisticRegression(C=2, class_weight='balanced')
lg.fit(X_train, Y_train)

LogisticRegression(C=2, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [102]:
((cv_res['split0_test_score'] + cv_res['split1_test_score'] + cv_res['split2_test_score'])/3).mean()

0.70815261148481934

In [103]:
roc_auc_score(Y_test, list(map(lambda x: x[1], lg.predict_proba(X_test))))

0.68626460017969459

In [40]:
from sklearn.metrics import precision_recall_curve

In [41]:
precision, recall, thresholds = precision_recall_curve(Y_test, list(map(lambda x: x[1], lg.predict_proba(X_test))))

In [137]:
from sklearn.metrics import precision_recall_fscore_support as prf

precision,recall,fscore,_ = prf(Y_test, lg.predict(X_test))